# Coleta IRFCL

In [1]:
core_indicators = [
    # 1) Nível e composição das reservas
    "IRFCLDT1_IRFCL65_USD_IRFCL13",      # Reservas oficiais (total)
    "IRFCLDT1_IRFCL54_USD_IRFCL13",      # Reservas oficiais + outros FX assets
    "IRFCLDT1_IRFCL56_USD_IRFCL13",      # Ouro
    "IRFCLDT1_IRFCL57_USD_IRFCL13",      # Posição de reservas no FMI
    "IRFCLDT1_IRFCL65_DIC_XDR_USD_IRFCL13",  # SDR (holdings) dentro das reservas
    "IRFCLDT1_IRFCL32_USD_IRFCL13",      # Títulos nas reservas
    "IRFCLDT1_IRFCLCDCFC_USD_IRFCL13",   # Moeda e depósitos nas reservas

    # 2) Drenagens de curto prazo (headline + buckets)
    "IRFCLDT2_USD_IRFCL13",                          # Drenagens líquidas de curto prazo (total)
    "IRFCLDT2_IRFCL24_SUTM_USD_IRFCL13",             # <= 1 mês (loans/securities/deposits)
    "IRFCLDT2_IRFCL24_SM1MUT3M_USD_IRFCL13",         # 1–3 meses
    "IRFCLDT2_IRFCL24_SM3MUTY_USD_IRFCL13",          # 3–12 meses

    # 2.1) Quebra por principal/juros (saídas)
    "IRFCLDT2_IRFCL26_SUTM_FO_USD_IRFCL13",          # Principal <= 1 mês
    "IRFCLDT2_IRFCL26_SM1MUT3M_FO_USD_IRFCL13",      # Principal 1–3 meses
    "IRFCLDT2_IRFCL26_SM3MUTY_FO_USD_IRFCL13",       # Principal 3–12 meses
    "IRFCLDT2_IRFCL151_SUTM_FO_USD_IRFCL13",         # Juros <= 1 mês
    "IRFCLDT2_IRFCL151_SM1MUT3M_FO_USD_IRFCL13",     # Juros 1–3 meses
    "IRFCLDT2_IRFCL151_SM3MUTY_FO_USD_IRFCL13",      # Juros 3–12 meses

    # 3) Derivativos / forwards (impacto na liquidez)
    "IRFCLDT2_IRFCL1_SUTM_IN_LP_USD_IRFCL13",        # Forwards/Futuros: entradas <= 1 mês
    "IRFCLDT2_IRFCL1_SUTM_SHP_USD_IRFCL13",          # Forwards/Futuros: saídas <= 1 mês
    "IRFCLDT4_IRFCLU97_A_USD_IRFCL13",               # Derivativos (net MTM) - memorando

    # 4) Memorando: composição por moeda e títulos em repo
    "IRFCLDT4_IRFCL11_DIC_XDRB_USD_IRFCL13",         # Cesta SDR (por moeda)
    "IRFCLDT4_IRFCL11_DIC_XXDR_USD_IRFCL13",         # Fora da cesta SDR
    "IRFCLDT4_IRFCL68_USD_IRFCL13",                  # Títulos cedidos/em repo
    "IRFCLDT4_IRFCL69X_USD_IRFCL13",                 # ...não incluídos na seção I
]

In [2]:
important_countries = [
    # Maiores economias / benchmarks
    "USA","CHN","DEU","JPN","GBR","FRA","ITA","IND","CAN","AUS","KOR","ESP","NLD",
    # Europa ampliada
    "IRL","CHE","LUX","SWE","DNK","FIN","POL","AUT","BEL","CZE","HUN","ROU","PRT","GRC",
    # Américas (desenvolvidos + EM)
    "BRA","MEX","ARG","COL","CHL","PER","URY",
    # Ásia emergente e manufatura
    "TWN","SGP","HKG","THA","MYS","VNM","IDN","PHL",
    # Oriente Médio / petróleo
    "SAU","ARE","QAT","KWT","IRN","ISR",
    # Commodities avançados / energia
    "NOR","RUS",
    # África (representativos)
    "ZAF","EGY","NGA","GHA","KEN",
]

In [3]:
import sdmx
import pandas as pd

nome_base = 'IRFCL'


# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

df_final = pd.DataFrame() 


# Abrir o arquivo para adicionar conteúdo (append)
with open("1_DICIONARIO.txt", "a", encoding="utf-8") as f:

    for country in important_countries:
        f.write("----------------------\n\n")
        for i,core_indicator in enumerate(core_indicators):
            # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
            key = dict(
                COUNTRY = country,
                # BOP_ACCOUNTING_ENTRY = ac,
                INDICATOR = core_indicator,
                FREQUENCY = "Q",
                # UNIT= "USDB"
            )

            # Baixar os dados
            data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

            # Converter para pandas DataFrame
            df = sdmx.to_pandas(
                data_msg, 
                #datetime="TIME_PERIOD"
                )

            df = df.reset_index()

            df_final = pd.concat([df_final, df], ignore_index=True)
            if not df.empty:
                print(f"País {country},indicador: {core_indicator}")
                f.write(f"\nPaís {country},indicador: {core_indicator}")
            print(f"Finalizada etapa {i}") if i == 1 else print




display(df_final.head())
df_final.to_csv(f"/opt/airflow/Resultados/{nome_base}.csv", index=False)

País USA,indicador: IRFCLDT1_IRFCL65_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL54_USD_IRFCL13
Finalizada etapa 1
País USA,indicador: IRFCLDT1_IRFCL56_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL57_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL65_DIC_XDR_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL32_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCLCDCFC_USD_IRFCL13
País USA,indicador: IRFCLDT2_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SUTM_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SM1MUT3M_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SM3MUTY_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SUTM_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SM1MUT3M_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SM3MUTY_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SUTM_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SM1MUT3M_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SM3MUTY_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL1_SUTM_IN_LP_USD_IRFCL13

,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value,index
0,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2000-Q4,6.693000e+10,NaN
1,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q1,6.371900e+10,NaN
2,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q2,6.435500e+10,NaN
3,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q3,7.038200e+10,NaN
4,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q4,6.862100e+10,NaN
